In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

# MULTIPROCESSING DDP STUFF
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
from torch.nn.parallel import DistributedDataParallel as DDP

import importlib
import os

# Try to import model
import sys
sys.path.insert(0, '/cephyr/NOBACKUP/groups/globalpoverty1/JesperBenjamin/geography-aware-ssl/moco_fmow/moco')
from ms_nl_resnet import MS_NL_ResNet18

import supervised_dataset

importlib.reload(supervised_dataset)

<module 'supervised_dataset' from '/cephyr/NOBACKUP/groups/globalpoverty1/JesperBenjamin/Supervised/supervised_dataset.py'>

In [10]:
# MODEL STUFF
#from tensorboardX import SummaryWriter
import time

train_set = supervised_dataset.SupervisedDataset('dhs_clusters_paths_small.csv')#, print_times=True)
train_loader = torch.utils.data.DataLoader(
        train_set, batch_size=6,
        drop_last=True
)

#model = MS_NL_ResNet18(num_classes=1)
#print("CUDA: ", torch.cuda.is_available())
#
#gpu = 0

criterion = torch.nn.MSELoss().cuda(gpu)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

############
#torch.cuda.set_device(gpu)
#model.cuda(gpu)


# DISTRiBUTED
#dist.init_process_group(backend='nccl', init_method='tcp://127.0.0.1:8893',
#                                world_size=1, rank=0)

#model.cuda()
#model = torch.nn.parallel.DistributedDataParallel(model)

#train_sampler = torch.utils.data.distributed.DistributedSampler(
#            train_dataset)
print()

losses = [[] for x in range(100)]


In [3]:
# MULTIPROCESSING DDP STUFF
gpu = 0
rank = 0
world_size = 1

dist.init_process_group("nccl", init_method='tcp://127.0.0.1:8893', rank=rank, world_size=world_size)



In [5]:
model = MS_NL_ResNet18(num_classes=1)
torch.cuda.set_device(gpu)
model.cuda(gpu)

ddp_model = DDP(model, device_ids=[rank])

In [8]:
criterion = torch.nn.MSELoss().cuda(gpu)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)

In [25]:
def train(rank, world_size):
    print("asd1")
    for i, (images, labels) in enumerate(train_loader):
        images = images.cuda(gpu)
        labels = labels.cuda(gpu)
        
        print("asd2")
        output = ddp_model(images)

        print(output)
        
        textfile = open("asd.txt", "w")
        textfile.write("asd")
        textfile.close()
        
        break

In [27]:
mp.spawn(train, args=(world_size,), nprocs = world_size,
        join=True)

ProcessExitedException: process 0 terminated with exit code 1